In [1]:
library(readr)
library(dplyr)



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [11]:
library("VariantAnnotation")

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min


Loading req

In [3]:
a = read_delim("/mnt/mfs/statgen//neuro-apex/demo_with_mwe/cis/MWE.21.cis_long_table.txt.gz","\t") -> a


── Column specification ────────────────────────────────────────────────────────
cols(
  `#chrom` = col_double(),
  pos = col_double(),
  ref = col_character(),
  alt = col_character(),
  gene = col_character(),
  beta = col_double(),
  se = col_double(),
  pval = col_double()
)




In [4]:
a = a%>%dplyr::filter(alt != "*", ref != "*")

In [12]:
       create_vcf = function (chrom, pos, nea, ea, snp = NULL, ea_af = NULL, effect = NULL, 
        se = NULL, pval = NULL, n = NULL, ncase = NULL, name = NULL,cs = NULL, pip = NULL) 
    {
        stopifnot(length(chrom) == length(pos))
        if (is.null(snp)) {
            snp <- paste0(chrom, ":", pos)
        }
        snp <- paste0(chrom, ":", pos)
        nsnp <- length(chrom)
        gen <- list()
        ## Setupt data content for each sample column
        if (!is.null(ea_af)) 
            gen[["AF"]] <- matrix(ea_af, nsnp)
        if (!is.null(effect)) 
            gen[["ES"]] <- matrix(effect, nsnp)
        if (!is.null(se)) 
            gen[["SE"]] <- matrix(se, nsnp)
        if (!is.null(pval)) 
            gen[["LP"]] <- matrix(-log10(pval), nsnp)
        if (!is.null(n)) 
            gen[["SS"]] <- matrix(n, nsnp)
        if (!is.null(ncase)) 
            gen[["NC"]] <- matrix(ncase, nsnp)
        gen <- S4Vectors::SimpleList(gen)
        
      ## Setup snps info for the fix columns
        gr <- GenomicRanges::GRanges(chrom, IRanges::IRanges(start = pos, 
            end = pos + pmax(nchar(nea), nchar(ea)) - 1, names = snp))
         coldata <- S4Vectors::DataFrame(Studies = name, row.names = name)
    ## Setup header informations
        hdr <- VariantAnnotation::VCFHeader(header = IRanges::DataFrameList(fileformat = S4Vectors::DataFrame(Value = "VCFv4.2", 
            row.names = "fileformat")), sample = name)
        VariantAnnotation::geno(hdr) <- S4Vectors::DataFrame(Number = c("A", 
            "A", "A", "A", "A", "A"), Type = c("Float", "Float", 
            "Float", "Float", "Float", "Float"), Description = c("Effect size estimate relative to the alternative allele", 
            "Standard error of effect size estimate", "-log10 p-value for effect estimate", 
            "Alternate allele frequency in the association study", 
            "Sample size used to estimate genetic effect", "Number of cases used to estimate genetic effect"), 
            row.names = c("ES", "SE", "LP", "AF", "SS", "NC"))
      ## Save only the meta information in the sample columns 
        VariantAnnotation::geno(hdr) <- subset(VariantAnnotation::geno(hdr), 
            rownames(VariantAnnotation::geno(hdr)) %in% names(gen))
      ## Save VCF 
        vcf <- VariantAnnotation::VCF(rowRanges = gr, colData = coldata, 
            exptData = list(header = hdr), geno = gen)
        VariantAnnotation::alt(vcf) <- Biostrings::DNAStringSetList(as.list(ea))
        VariantAnnotation::ref(vcf) <- Biostrings::DNAStringSet(nea)
      ## Add fixed values
        VariantAnnotation::fixed(vcf)$FILTER <- "PASS"
      ## Add Info values
        if (!is.null(cs)) {
        VariantAnnotation::info(vcf)$CS <- cs
            }
        if (!is.null(pip)) {
        VariantAnnotation::info(vcf)$PIP <- pip
            }
      ## Add header for info
        VariantAnnotation::info(header(vcf)) <- S4Vectors::DataFrame(Number = c("A", 
            "A"), Type = c("Float", "Float"), Description = c("The credible set this snv located in", 
            "The posterior inclusion probability for this snv in the credible set"), 
            row.names = c("CS","PIP"))
        #VariantAnnotation::info(hdr) <- subset(VariantAnnotation::info(hdr), 
         #   rownames(VariantAnnotation::info(hdr)) %in% names(info(vcf)))
           return(sort(vcf))
        }

In [5]:
names(info((b)))

ERROR: Error in info((b)): could not find function "info"


In [65]:
 hdr <- VariantAnnotation::VCFHeader(header = IRanges::DataFrameList(fileformat = S4Vectors::DataFrame(Value = "VCFv4.2", 
            row.names = "fileformat")), sample = "TTT")

In [70]:
info(hdr)

DataFrame with 2 rows and 3 columns
         Number        Type
    <character> <character>
CS            A       Float
PIP           A       Float
                                                             Description
                                                             <character>
CS                                  The credible set this snv located in
PIP The posterior inclusion probability for this snv in the credible set

In [79]:
 VariantAnnotation::info(header(b)) <- S4Vectors::DataFrame(Number = c("A", 
            "A"), Type = c("Float", "Float"), Description = c("The credible set this snv located in", 
            "The posterior inclusion probability for this snv in the credible set"), 
            row.names = c("CS","PIP"))
sort(b)

class: CollapsedVCF 
dim: 102064 1 
rowRanges(vcf):
  GRanges with 4 metadata columns: REF, ALT, QUAL, FILTER
info(vcf):
  DataFrame with 2 columns: CS, PIP
info(header(vcf)):
       Number Type  Description                                                
   CS  A      Float The credible set this snv located in                       
   PIP A      Float The posterior inclusion probability for this snv in the ...
geno(vcf):
  List of length 3: ES, SE, LP
geno(header(vcf)):
      Number Type  Description                                            
   ES A      Float Effect size estimate relative to the alternative allele
   SE A      Float Standard error of effect size estimate                 
   LP A      Float -log10 p-value for effect estimate                     

In [77]:
b

class: CollapsedVCF 
dim: 102064 1 
rowRanges(vcf):
  GRanges with 4 metadata columns: REF, ALT, QUAL, FILTER
info(vcf):
  DataFrame with 2 columns: CS, PIP
info(header(vcf)):
       Number Type  Description                                                
   CS  A      Float The credible set this snv located in                       
   PIP A      Float The posterior inclusion probability for this snv in the ...
geno(vcf):
  List of length 3: ES, SE, LP
geno(header(vcf)):
      Number Type  Description                                            
   ES A      Float Effect size estimate relative to the alternative allele
   SE A      Float Standard error of effect size estimate                 
   LP A      Float -log10 p-value for effect estimate                     

In [78]:
d = create_vcf(
   chrom = a$`#chrom`,
    pos = a$pos,
    ea = a$ref,
    nea = a$alt,
  effect = a$beta,
  se = a$se,
  pval = a$pval,
    name = "TTT", cs = a$pval , pip =  a$pval )



Warning message in matrix(-log10(pval), nsnp):
“NaNs produced”
Warning message:
“info fields with no header: CS”
Warning message:
“info fields with no header: CS,PIP”


In [10]:
d

ERROR: Error in eval(expr, envir, enclos): object 'd' not found


In [63]:
class(info(header(b)))

[1] "DFrame"
attr(,"package")
[1] "S4Vectors"

In [39]:
VariantAnnotation::info(d)$TEST = a$pval

Warning message:
“info fields with no header: TEST”


In [8]:
d

ERROR: Error in eval(expr, envir, enclos): object 'd' not found


In [40]:
b
VariantAnnotation::writeVcf(d,"~/test.vcf",index = TRUE)

class: CollapsedVCF 
dim: 102064 1 
rowRanges(vcf):
  GRanges with 4 metadata columns: REF, ALT, QUAL, FILTER
info(vcf):
  DataFrame with 1 column: TEST
  Fields with no header: TEST 
geno(vcf):
  List of length 3: ES, SE, LP
geno(header(vcf)):
      Number Type  Description                                            
   ES A      Float Effect size estimate relative to the alternative allele
   SE A      Float Standard error of effect size estimate                 
   LP A      Float -log10 p-value for effect estimate                     

In [13]:
d = create_vcf(
   chrom = a$`#chrom`,
    pos = a$pos,
    ea = a$ref,
    nea = a$alt,
  effect = cbind(a$se,a$beta+1),
    pval = cbind(a$se,a$beta+1),
    name = c("AC","DLPFC_PCC"))


In [14]:
Result

class: CollapsedVCF 
dim: 102064 2 
rowRanges(vcf):
  GRanges with 4 metadata columns: REF, ALT, QUAL, FILTER
info(vcf):
  DataFrame with 0 columns: 
geno(vcf):
  List of length 2: ES, LP
geno(header(vcf)):
      Number Type  Description                                            
   ES A      Float Effect size estimate relative to the alternative allele
   LP A      Float -log10 p-value for effect estimate                     

[1] 102064      2

In [35]:
create_vcf

function (chrom, pos, nea, ea, snp = NULL, ea_af = NULL, effect = NULL, 
    se = NULL, pval = NULL, n = NULL, ncase = NULL, name = NULL) 
{
    stopifnot(length(chrom) == length(pos))
    if (is.null(snp)) {
        snp <- paste0(chrom, ":", pos)
    }
    nsnp <- length(chrom)
    gen <- list()
    if (!is.null(ea_af)) 
        gen[["AF"]] <- matrix(ea_af, nsnp)
    if (!is.null(effect)) 
        gen[["ES"]] <- matrix(effect, nsnp)
    if (!is.null(se)) 
        gen[["SE"]] <- matrix(se, nsnp)
    if (!is.null(pval)) 
        gen[["LP"]] <- matrix(-log10(pval), nsnp)
    if (!is.null(n)) 
        gen[["SS"]] <- matrix(n, nsnp)
    if (!is.null(ncase)) 
        gen[["NC"]] <- matrix(ncase, nsnp)
    gen <- S4Vectors::SimpleList(gen)
    gr <- GenomicRanges::GRanges(chrom, IRanges::IRanges(start = pos, 
        end = pos + pmax(nchar(nea), nchar(ea)) - 1, names = snp))
    coldata <- S4Vectors::DataFrame(Samples = length(name), row.names = name)
    hdr <- VariantAnnotation::VCFHeader(header = IRanges::DataFrameList(fileformat = S4Vectors::DataFrame(Value = "VCFv4.2", 
        row.names = "fileformat")), sample = name)
    VariantAnnotation::geno(hdr) <- S4Vectors::DataFrame(Number = c("A", 
        "A", "A", "A", "A", "A"), Type = c("Float", "Float", 
        "Float", "Float", "Float", "Float"), Description = c("Effect size estimate relative to the alternative allele", 
        "Standard error of effect size estimate", "-log10 p-value for effect estimate", 
        "Alternate allele frequency in the association study", 
        "Sample size used to estimate genetic effect", "Number of cases used to estimate genetic effect"), 
        row.names = c("ES", "SE", "LP", "AF", "SS", "NC"))
    VariantAnnotation::geno(hdr) <- subset(VariantAnnotation::geno(hdr), 
        rownames(VariantAnnotation::geno(hdr)) %in% names(gen))
    vcf <- VariantAnnotation::VCF(rowRanges = gr, colData = coldata, 
        exptData = list(header = hdr), geno = gen)
    VariantAnnotation::alt(vcf) <- Biostrings::DNAStringSetList(as.list(ea))
    VariantAnnotation::ref(vcf) <- Biostrings::DNAStringSet(nea)
    VariantAnnotation::fixed(vcf)$FILTER <- "PASS"
    return(sort(vcf))
}
<bytecode: 0x558c86492638>
<environment: namespace:gwasvcf>

In [40]:
effect = cbind(a$se,a$beta+1)

In [42]:
matrix(effect,nrow(effect))

0.02678740,1.0045513
0.00863702,1.0191654
0.07035390,1.0317659
0.07023000,1.0879099
0.07042450,0.9787639
0.00561773,1.0015735
0.00830861,0.9954964
0.00400209,0.9949001
0.00441183,0.9912124
0.02500750,0.9926339
0.01431390,1.0034420


In [27]:
rowRanges(k) = rowRanges(k)[,1:4]

In [160]:
merge_vcf(b,d)

ERROR: Error in .::S4Vectors: unused argument (SimpleList)


In [135]:
a = d

In [ ]:
cbind(b,d)-> k

In [136]:
a <- VariantAnnotation::expand(a)
    b <- VariantAnnotation::expand(b)
    o <- dplyr::tibble(from = which(names(a) %in% names(b)), 
        to = match(names(a)[.data$from], names(b)))
    a <- a[o[["from"]], ]
    b <- b[o[["to"]], ]
    allele_match <- VariantAnnotation::ref(a) == VariantAnnotation::ref(b) & 
        VariantAnnotation::alt(a) == VariantAnnotation::alt(b)
    switch <- VariantAnnotation::ref(a) == VariantAnnotation::alt(b) & 
        VariantAnnotation::ref(b) == VariantAnnotation::alt(a)
    if (any(switch)) {
        for (i in 1:ncol(VariantAnnotation::geno(b)[["ES"]])) {
            VariantAnnotation::geno(b)[["ES"]][, i][switch] <- lapply(VariantAnnotation::geno(b)[["ES"]][, 
                i][switch], function(x) x * -1)
        }
    }
    a <- a[allele_match | switch, ]
    b <- b[allele_match | switch, ]
    ab <- a


ERROR: Error in all_dims[, 1L]: incorrect number of dimensions


In [143]:
sum(switch)

[1] 78900

In [71]:
    temp <- lapply(names(VariantAnnotation::geno(ab)), function(x) rbind(VariantAnnotation::geno(ab)[[x]], 
        VariantAnnotation::geno(b)[x])) %>% S4Vectors::SimpleList()
    names(temp) <- names(VariantAnnotation::geno(ab))


In [122]:
temp <- lapply(names(VariantAnnotation::geno(ab)), function(x) rbind(VariantAnnotation::geno(ab)[[x]], 
        VariantAnnotation::geno(b)[[x]]))%>%SimpleList

In [123]:
temp

List of length 3

In [161]:
d

function (a, b) 
{
    a <- VariantAnnotation::expand(a)
    b <- VariantAnnotation::expand(b)
    o <- dplyr::tibble(from = which(names(a) %in% names(b)), 
        to = match(names(a)[.data$from], names(b)))
    a <- a[o[["from"]], ]
    b <- b[o[["to"]], ]
    allele_match <- VariantAnnotation::ref(a) == VariantAnnotation::ref(b) & 
        VariantAnnotation::alt(a) == VariantAnnotation::alt(b)
    switch <- VariantAnnotation::ref(a) == VariantAnnotation::alt(b) & 
        VariantAnnotation::ref(b) == VariantAnnotation::alt(a)
    if (any(switch)) {
        for (i in 1:ncol(VariantAnnotation::geno(b)[["ES"]])) {
            VariantAnnotation::geno(b)[["ES"]][, i][switch] <- lapply(VariantAnnotation::geno(b)[["ES"]][, 
                i][switch], function(x) x * -1)
        }
    }
    a <- a[allele_match | switch, ]
    b <- b[allele_match | switch, ]
    ab <- a
    temp <- lapply(names(VariantAnnotation::geno(ab)), function(x) rbind(VariantAnnotation::geno(ab)[x], 
        VariantAnnotation::geno(b)[x])) %>% S4Vectors::SimpleList
    names(temp) <- names(VariantAnnotation::geno(ab))
    VariantAnnotation::geno(ab) <- temp
    h <- VariantAnnotation::header(a)
    VCFHeader(reference = VariantAnnotation::reference(h), samples = c(VariantAnnotation::samples(h), 
        VariantAnnotation::samples(VariantAnnotation::header(b))), 
        meta = VariantAnnotation::meta(h))
    VariantAnnotation::samples(h) <- c(VariantAnnotation::samples(h), 
        VariantAnnotation::samples(VariantAnnotation::header(b)))
    return(S4Vectors::SimpleList())
}
<bytecode: 0x55a09e013a88>
<environment: namespace:gwasvcf>

In [172]:
merge_vcf = function (a, b) 
{
    a <- VariantAnnotation::expand(a)
    b <- VariantAnnotation::expand(b)
    o <- dplyr::tibble(from = which(names(a) %in% names(b)), 
        to = match(names(a)[.data$from], names(b)))
    a <- a[o[["from"]], ]
    b <- b[o[["to"]], ]
    allele_match <- VariantAnnotation::ref(a) == VariantAnnotation::ref(b) & 
        VariantAnnotation::alt(a) == VariantAnnotation::alt(b)
    switch <- VariantAnnotation::ref(a) == VariantAnnotation::alt(b) & 
        VariantAnnotation::ref(b) == VariantAnnotation::alt(a)
    if (any(switch)) {
        for (i in 1:ncol(VariantAnnotation::geno(b)[["ES"]])) {
            VariantAnnotation::geno(b)[["ES"]][, i][switch] <- lapply(VariantAnnotation::geno(b)[["ES"]][, 
                i][switch], function(x) x * -1)
        }
    }
    a <- a[allele_match | switch, ]
    b <- b[allele_match | switch, ]
    ab <- a
    temp <- lapply(names(VariantAnnotation::geno(ab)), function(x) cbind(VariantAnnotation::geno(ab)[[x]], 
        VariantAnnotation::geno(b)[[x]])) %>% S4Vectors::SimpleList()
    names(temp) <- names(VariantAnnotation::geno(ab))
    VariantAnnotation::geno(ab) <- temp
    h <- VariantAnnotation::header(a)
    VCFHeader(reference = VariantAnnotation::reference(h), samples = c(VariantAnnotation::samples(h), 
        VariantAnnotation::samples(VariantAnnotation::header(b))), 
        meta = VariantAnnotation::meta(h))
    VariantAnnotation::samples(h) <- c(VariantAnnotation::samples(h), 
        VariantAnnotation::samples(VariantAnnotation::header(b)))
    return(S4Vectors::SimpleList())
}

In [74]:
    VariantAnnotation::geno(ab) <- temp

List of length 3
names(3): ES SE LP

In [96]:
tet = temp
tet$ES = temp$ES[[1]]
tet$SE = temp$SE[[1]]
tet$LP = temp$LP[[1]]

In [129]:
g

,TTT
21:30873191,0.004551320
21:30873237,0.019165400
21:30873308,0.031765900
21:30873319,0.087909900
21:30873432,-0.021236100
21:30873441,0.001573480
21:30873445,-0.004503620
21:30873446,-0.005099940
21:30873447,-0.008787590
21:30873449,-0.007366060


In [85]:
d

class: CollapsedVCF 
dim: 102064 1 
rowRanges(vcf):
  GRanges with 4 metadata columns: REF, ALT, QUAL, FILTER
info(vcf):
  DataFrame with 0 columns: 
geno(vcf):
  List of length 3: ES, SE, LP
geno(header(vcf)):
      Number Type  Description                                            
   ES A      Float Effect size estimate relative to the alternative allele
   SE A      Float Standard error of effect size estimate                 
   LP A      Float -log10 p-value for effect estimate                     

In [59]:
c = vcf_to_tibble(b)

ERROR: Error in parse(text = x, srcfile = src): <text>:2:0: unexpected end of input
1: d = 
   ^


In [50]:

   chrom = a$`#chrom`
    pos = a$pos
    ea = a$ref
    nea = a$alt
  effect = cbind(a$se,a$beta+1)
  se =  cbind(a$se,a$beta+1)
    pval = cbind(a$se,a$beta+1)
    name = c("sss","qqq")

In [58]:
ea_af = NULL
n = NULL
ncase = NULL

In [37]:
create_vcf = function (chrom, pos, nea, ea, snp = NULL, ea_af = NULL, effect = NULL, 
    se = NULL, pval = NULL, n = NULL, ncase = NULL, name = NULL) 
{
    stopifnot(length(chrom) == length(pos))
    if (is.null(snp)) {
        snp <- paste0(chrom, ":", pos)
    }
    snp <- paste0(chrom, ":", pos)
    nsnp <- length(chrom)
    gen <- list()
    if (!is.null(ea_af)) 
        gen[["AF"]] <- matrix(ea_af, nsnp)
    if (!is.null(effect)) 
        gen[["ES"]] <- matrix(effect, nsnp)
    if (!is.null(se)) 
        gen[["SE"]] <- matrix(se, nsnp)
    if (!is.null(pval)) 
        gen[["LP"]] <- matrix(-log10(pval), nsnp)
    if (!is.null(n)) 
        gen[["SS"]] <- matrix(n, nsnp)
    if (!is.null(ncase)) 
        gen[["NC"]] <- matrix(ncase, nsnp)
    gen <- S4Vectors::SimpleList(gen)
    gr <- GenomicRanges::GRanges(chrom, IRanges::IRanges(start = pos, 
        end = pos + pmax(nchar(nea), nchar(ea)) - 1, names = snp))
     coldata <- S4Vectors::DataFrame(Studies = name, row.names = name)
    hdr <- VariantAnnotation::VCFHeader(header = IRanges::DataFrameList(fileformat = S4Vectors::DataFrame(Value = "VCFv4.2", 
        row.names = "fileformat")), sample = name)
    VariantAnnotation::geno(hdr) <- S4Vectors::DataFrame(Number = c("A", 
        "A", "A", "A", "A", "A"), Type = c("Float", "Float", 
        "Float", "Float", "Float", "Float"), Description = c("Effect size estimate relative to the alternative allele", 
        "Standard error of effect size estimate", "-log10 p-value for effect estimate", 
        "Alternate allele frequency in the association study", 
        "Sample size used to estimate genetic effect", "Number of cases used to estimate genetic effect"), 
        row.names = c("ES", "SE", "LP", "AF", "SS", "NC"))
    VariantAnnotation::geno(hdr) <- subset(VariantAnnotation::geno(hdr), 
        rownames(VariantAnnotation::geno(hdr)) %in% names(gen))
    vcf <- VariantAnnotation::VCF(rowRanges = gr, colData = coldata, 
        exptData = list(header = hdr), geno = gen)
    VariantAnnotation::alt(vcf) <- Biostrings::DNAStringSetList(as.list(ea))
    VariantAnnotation::ref(vcf) <- Biostrings::DNAStringSet(nea)
    VariantAnnotation::fixed(vcf)$FILTER <- "PASS"
        return(sort(vcf))
    }

In [82]:
l$Studies

[1] "sss" "qqq"

In [75]:
info(l)

DataFrame with 102064 rows and 0 columns

In [ ]:
    coldata <- S4Vectors::DataFrame(Samples = length(name), row.names = name)
    hdr <- VariantAnnotation::VCFHeader(header = IRanges::DataFrameList(fileformat = S4Vectors::DataFrame(Value = "VCFv4.2", 
        row.names = "fileformat")), sample = name)
    VariantAnnotation::geno(hdr) <- S4Vectors::DataFrame(Number = c("A", 
        "A", "A", "A", "A", "A"), Type = c("Float", "Float", 
        "Float", "Float", "Float", "Float"), Description = c("Effect size estimate relative to the alternative allele", 
        "Standard error of effect size estimate", "-log10 p-value for effect estimate", 
        "Alternate allele frequency in the association study", 
        "Sample size used to estimate genetic effect", "Number of cases used to estimate genetic effect"), 
        row.names = c("ES", "SE", "LP", "AF", "SS", "NC"))
    VariantAnnotation::geno(hdr) <- subset(VariantAnnotation::geno(hdr), 
        rownames(VariantAnnotation::geno(hdr)) %in% names(gen))
    vcf <- VariantAnnotation::VCF(rowRanges = gr, colData = coldata, 
        exptData = list(header = hdr), geno = gen)
    VariantAnnotation::alt(vcf) <- Biostrings::DNAStringSetList(as.list(ea))
    VariantAnnotation::ref(vcf) <- Biostrings::DNAStringSet(nea)
    VariantAnnotation::fixed(vcf)$FILTER <- "PASS"

In [27]:

    snp <- paste0(chrom, ":", pos)
    nsnp <- length(chrom)
    gen <- list()

In [28]:
 if (!is.null(ea_af)) 
        gen[["AF"]] <- matrix(ea_af, nsnp)
    if (!is.null(effect)) 
        gen[["ES"]] <- matrix(effect, nsnp)
    if (!is.null(se)) 
        gen[["SE"]] <- matrix(se, nsnp)
    if (!is.null(pval)) 
        gen[["LP"]] <- matrix(-log10(pval), nsnp)
    if (!is.null(n)) 
        gen[["SS"]] <- matrix(n, nsnp)
    if (!is.null(ncase)) 
        gen[["NC"]] <- matrix(ncase, nsnp)

In [29]:
    gen <- S4Vectors::SimpleList(gen)
    gr <- GenomicRanges::GRanges(chrom, IRanges::IRanges(start = pos, 
        end = pos + pmax(nchar(nea), nchar(ea)) - 1, names = snp))
    coldata <- S4Vectors::DataFrame(Samples = length(name), row.names = name)

In [34]:
hdr <- VariantAnnotation::VCFHeader(header = IRanges::DataFrameList(fileformat = S4Vectors::DataFrame(Value = "VCFv4.2", 
        row.names = "fileformat")), sample = name)

In [36]:
VariantAnnotation::geno(hdr) <- subset(VariantAnnotation::geno(hdr), 
        rownames(VariantAnnotation::geno(hdr)) %in% names(gen))

In [35]:
VariantAnnotation::geno(hdr) <- S4Vectors::DataFrame(Number = c("A", 
        "A", "A", "A", "A", "A"), Type = c("Float", "Float", 
        "Float", "Float", "Float", "Float"), Description = c("Effect size estimate relative to the alternative allele", 
        "Standard error of effect size estimate", "-log10 p-value for effect estimate", 
        "Alternate allele frequency in the association study", 
        "Sample size used to estimate genetic effect", "Number of cases used to estimate genetic effect"), 
        row.names = c("ES", "SE", "LP", "AF", "SS", "NC"))

In [42]:
    VariantAnnotation::alt(vcf) <- Biostrings::DNAStringSetList(as.list(ea))
    VariantAnnotation::ref(vcf) <- Biostrings::DNAStringSet(nea)

ERROR: Error in .Call2("new_XStringSet_from_CHARACTER", class(x0), elementType(x0), : key 42 (char '*') not in lookup table


In [37]:
vcf <- VariantAnnotation::VCF(rowRanges = gr, colData = coldata, 
        exptData = list(header = hdr), geno = gen)

In [23]:
a[which(is.na(gen[["LP"]])),]

#chrom,pos,ref,alt,gene,beta,se,pval
<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
21,30873708,A,G,ENSG00000142149,0,0,-1
21,30874181,T,A,ENSG00000142149,0,0,-1
21,30874182,C,A,ENSG00000142149,0,0,-1
21,30874183,C,A,ENSG00000142149,0,0,-1
21,30874585,G,A,ENSG00000142149,0,0,-1
21,30874921,C,T,ENSG00000142149,0,0,-1
21,30875013,C,CT,ENSG00000142149,0,0,-1
21,30875332,T,C,ENSG00000142149,0,0,-1
21,30875460,T,G,ENSG00000142149,0,0,-1
